<a href="https://githubtocolab.com/kaust-halo/geeet/blob/master/examples/notebooks/03_eepredefined_landsat_era5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

# EE predefined: Landsat + ERA5 image collection

---

The `geeet.eepredefined` library contains a collection of modules that make it easy to ingest GEE data into the geeet models. 

This notebook demonstrates how to:

1. Prepare a [merged Landsat collection](#landsat-collection-02) (Landsat 7, 8, and 9) 
2. Prepare a [joint Landsat + ERA5](#era5-climate-reanalysis-data) (climate reanalysis) image collection
3. Map the Landsat + ERA5 collection [into an ET model](#landsat--era5--tseb) 

In [ ]:
# Use the following line to install geeet if needed:
#!pip install git+https://github.com/kaust-halo/geeet
# Use the following line to install geemap if needed:
#!pip install geemap
import ee
#ee.Authenticate() # Uncomment if using Google Colab or first time using EE on this device. 
ee.Initialize()

## Landsat collection 02

Prepare a merged Landsat (Collection 02) surface reflectance image collection using `landsat.collection`.

All surface reflectance (`SR_*`) and temperature (`ST_*`) bands are scaled and offset. The collection is also filtered by:

- date: using `date_start` and `date_end` 
- bounds: using `region` (could be a dict, string indicating an asset, local shapefile/geojson, or a [geopandas.GeoDataFrame](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.html))

In [ ]:
import geeet
from geeet.eepredefined import landsat

region = dict(type="Point", coordinates=[38.25, 30.25])

coll =  landsat.collection(
    date_start = "2023-04-01",
    date_end = "2023-05-01",
    region = region, 
)

Other filters in `landsat.collection` include: 

- Maximum cloud cover: Only include images with `CLOUD_COVER` less than or equal to `max_cc`. Use `max_cc` to set this value.  
- Explicitly exclude path/rows: Use `exclude_pr` to explicitly exclude a list of path/rows (e.g. `exclude_pr=[[171,39]]` will exclude images in the path/row 171/39). 
- Explicitly limit images to a list of path/rows: Use `include_pr` to filter the collection to *only* include images within a list of path/rows.
- Include *only* images from `sat_list`, which may be a **non-empty** subset of `["LANDSAT_7", "LANDSAT_8", "LANDSAT_9"]`. Defaults to using all three collections. 

> Since the collections are merged, the `system:index` loses its original value. However, you can still find in in a new `LANDSAT_INDEX` property:

In [ ]:
coll.aggregate_array("LANDSAT_INDEX").getInfo()

All other properties and original band names remain intact:

In [ ]:
coll.first().bandNames().getInfo()

Note however, that by default four new bands were added. These are:

- `NDVI` (normalized difference vegetation index): calculated using the corresponding NIR and red bands. Use `ndvi=False` to exclude this band.
- `albedo`: calculated using one of two parameterizations ([Liang, 2001](https://www.sciencedirect.com/science/article/pii/S0034425700002054) by default). Use `albedo=None` to exclude this band. 
- `radiometric_temperature`: This is a copy of either the `ST_B6` (Landsat 7) or `ST_10` band (Landsat 8 and 9). Use `rad_temp=False` to exclude this band.
- `cloud_cover`: A byte band indicating 1 for pixels identified as cloud/cloud shadows, and 0 otherwise. Use `cfmask=False` to exclude this band. 

> The images are *not* masked using `cloud_cover`. However, you can use the helper function `landsat.cfmask` to update the mask of the desired bands to mask. 

Other optional bands include:

- lai: Leaf area index; calculated using one of two methods. By default `lai=None`. 

## ERA5 climate reanalysis data

*geeet.eepredefined* also includes some tools to work with [ERA5 climate reanalysis](https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_LAND_HOURLY#bands) data, specifically the `LAND_HOURLY` product. 

The `landsat.collection` tool also includes an option to include this data, specifically adding the following bands:

- `surface_pressure`: surface pressure, in Pa
- `air_temperature`: air temperature, in K
- `wind_speed`: wind speed, in m/s (also included are the `u` and `v` components as separate bands).
- `solar_radiation`: downward shortwave radiation, in W/m² (also included is the hourly accumulated radiation (`surface_solar_radiation_downwards_hourly`) band, in J/m²).
- `thermal_radiation`: downward longwave radiation, in W/m² (also included is the hourly accumulated radiation (`surface_thermal_radiation_downwards_hourly`) band, in J/m²).

In [ ]:
coll_with_era5 = landsat.collection(
    date_start = "2023-04-01",
    date_end = "2023-05-01",
    region = region, 
    era5 = True,
    timeZone="Asia/Riyadh" # the time property will be set in local time (UTC +3 for this example)
)
coll_with_era5.first().bandNames().getInfo()

## Landsat + ERA5 + TSEB

This collection is ready to be mapped to an ET model:

In [ ]:
landsat_era5_tseb_collection = coll_with_era5.map(geeet.tseb.tseb_series)

In [ ]:
landsat_era5_tseb_collection.first().bandNames().getInfo()

You can see now that the ET model outputs have been added as additional bands. 

You can also use the `landsat.mapped_collection` helper function to map a sequence of functions to a `landsat.collection`, e.g. the example above would become:

```python
landsat_era5_tseb_collection = landsat.mapped_collection(
    [geeet.tseb.tseb_series],    # List of mappable functions, here only tseb. 
    date_start = "2023-04-01",
    date_end = "2023-05-01",
    region = region, 
    era5 = True, 
    timeZone = "Asia/Riyadh"
)
```


## Visualization

Let's generate a small visualization by clipping one of the images to a small region. 

> You will need [geemap](geemap.org) for this visualization. Alternatively, open [this script in the code editor](https://code.earthengine.google.com/f71913e4516d36de0da2c0bce4f03d39). 

In [ ]:
import geemap
import geemap.colormaps as cm

# Select one image by LANDSAT_INDEX: 
image = (
    landsat_era5_tseb_collection
    .filter(ee.Filter.eq("LANDSAT_INDEX", 
    "LC09_172039_20230421"
    ))
    .first()
    .clip(ee.Geometry(region)
        .buffer(10000)   # 10km area around the point
        .bounds()        
    )
)

Map = geemap.Map()
Map.setCenter(*region["coordinates"], zoom=12)
ndvi_pal = cm.palettes.ndvi
red_pal = cm.palettes.YlOrRd
blue_pal = cm.palettes.YlGnBu

Map.addLayer(image, {'bands':['SR_B4', 'SR_B3', 'SR_B2'], 'min':0, 'max': 0.3}, 'Image')
Map.addLayer(image.select('albedo'), {'min':0, 'max':0.35}, 'Albedo (shortwave; Liang 2001)', False)
Map.addLayer(image.select('NDVI'), {'min':0, 'max':1, 'palette':ndvi_pal, 'opacity': 0.8}, 'NDVI')
Map.addLayer(image.select('radiometric_temperature'), {'min':20+273, 'max':50+273, 'palette':red_pal}, 'Landsat radiometric temperature (K)')
Map.addLayer(image.select('Rn'), {'min':0, 'max':500, 'palette':red_pal}, 'Net radiation (W/m²)', False)
Map.addLayer(image.select('LEc'), {'min':0, 'max':300, 'palette':blue_pal}, 'Latent heat flux (W/m²) from the canopy source', True)

Map

Here's a preview of how the map should look like:

![image](https://github.com/kaust-halo/geeet/assets/14804652/8379c468-1264-47ff-bdfb-89a7f9d8f61b)

## Export

This cell was used to export the image as an asset. 

In [ ]:
asset_id =  ""  # replace with your asset

roi = (ee.Geometry(region)
        .buffer(10000)   # 10km area around the point
        .bounds()        
)
task = ee.batch.Export.image.toAsset(image=image,
                                     description='landsat-era5-sample',
                                     assetId=asset_id,
                                     region=roi,
                                     crs="EPSG:32637",
                                     crsTransform=[30,0,296685,0,-30,3470115],
                                     )
#task.start() # Uncomment to submit the task.